In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [3]:
# read BPIC12.csv and Add label
# label = {'Activity' : 'O_ACCEPTED-COMPLETE'}

df = pd.read_csv('BPIC12.csv')
df = df[['Case ID', 'Activity', 'Resource', 'Complete Timestamp']]
df['label'] = ""
df.loc[df['Activity'] == 'O_ACCEPTED-COMPLETE', 'label'] = '1'
df.loc[df['Activity'] != 'O_ACCEPTED-COMPLETE', 'label'] = '0'
df

,Case ID,Activity,Resource,Complete Timestamp,label
0,173688,A_SUBMITTED-COMPLETE,112.0,2011/10/01 07:38:44.546,0
1,173688,A_PARTLYSUBMITTED-COMPLETE,112.0,2011/10/01 07:38:44.880,0
2,173688,A_PREACCEPTED-COMPLETE,112.0,2011/10/01 07:39:37.906,0
3,173688,W_Completeren aanvraag-SCHEDULE,112.0,2011/10/01 07:39:38.875,0
4,173688,W_Completeren aanvraag-START,NaN,2011/10/01 18:36:46.437,0
...,...,...,...,...,...
262195,214376,A_PARTLYSUBMITTED-COMPLETE,112.0,2012/03/01 07:51:17.423,0
262196,214376,W_Afhandelen leads-SCHEDULE,112.0,2012/03/01 07:52:01.287,0
262197,214376,W_Afhandelen leads-START,11169.0,2012/03/01 17:26:46.736,0
262198,214376,A_DECLINED-COMPLETE,11169.0,2012/03/01 17:27:37.118,0


In [ ]:
# dict : drop_act, bucketing, encoding, model

# drop_act = {2, 4, 6, 8}
# bucketing = {1, 2*mean_trace_length}
# encoding = {'aggregate', 'index'}
# model = {'DT', 'RF', 'XGB', 'LGBM'}

parameter = {'drop_act':[2,4,6,8], 
    'bucketing':[1, 2*'mean_trace_length'], # need to modify
    'encoding':['aggregate', 'index'],
    'model':['DT', 'RF', 'XGB', 'LGBM']}

In [18]:
# train test split

X = df.drop('label', axis=1)
y = df['label']

# one-hot encoding
# train and test set for decision tree
df_train = pd.get_dummies(data = X, columns = ['Activity'], prefix = 'Activity')
df_train['Complete Timestamp'] = pd.to_datetime(df_train['Complete Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')

# datetime to float for DT
df_train['Complete Timestamp'] = (df_train['Complete Timestamp'] - df_train['Complete Timestamp'].min()) / np.timedelta64(1,'D')
df_train

# remove NaN for DT
df_train = df_train.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=2023)

In [19]:
# machine learning
# DT
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df_train, y)
clf

DecisionTreeClassifier()

In [ ]:
# RF


# XGB

# LGBM